In [1]:
import numpy as np
import pandas as pd
from gensim.summarization.summarizer import summarize
from nltk import tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

import nucleus_api
from nucleus_api.rest import ApiException
import nucleus_api.api.nucleus_api as nucleus_helper
import csv

configuration = nucleus_api.Configuration()
configuration.host = 'http://migration.sumupanalytics.com:5000'
configuration.api_key['x-api-key'] = 'gj6m0H1pSvdw2YUFF5d4bQ'

# configuration.api_key['x-api-key'] = '3yGmNzWXX4oa1Xs539UXoA'
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [31]:
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
# from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import string
import matplotlib.pyplot as plt

def cleanup(doc):
    result = []
    doc = re.sub(re.compile('<.*?>'), '', doc)
    for word in gensim.utils.simple_preprocess(doc):
        if word not in gensim.parsing.preprocessing.STOPWORDS and len(doc) > 3:
            result.append(WordNetLemmatizer().lemmatize(word, pos='v'))
    return result

def apply_lda(data, num_topics):
    dictionary = gensim.corpora.Dictionary(data)
#     dictionary.filter_extremes(no_below=10, no_above=0.4)
    bow_corpus = [dictionary.doc2bow(text_lst) for text_lst in data]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=2, workers=4, iterations=10)
    distributions = [dict(lda_model[sentence]) for sentence in bow_corpus]
    return lda_model, distributions

# Inspect the dataset

In [2]:
data = pd.read_csv("philippine_earthquake_daily.csv")
data.head()

,time,title,content
0,2019-04-22,Strong Earthquake Shakes Areas North of Philip...,A strong earthquake shook the Philippine capit...
1,2019-04-22,"At least three dead due to quake, Pampanga gov...",The earthquake that jolted Luzon Monday aftern...
2,2019-04-22,6.1-magnitude earthquake in Philippine main Lu...,At least six people were killed after a 6.1-ma...
3,2019-04-22,Phivolcs: No technology yet to predict earthqu...,The Philippine Institute of Volcanology and Se...
4,2019-04-22,Earthquake of 6.3 magnitude strikes central Ph...,Walls move in circular fashion' as strong temb...


In [153]:
all_sents = tokenize.sent_tokenize(". ".join(list(data["content"]))) 

sents_bydate = {}
for date in data.time.unique():
    text = ". ".join(list(data["content"][data.time == date]))
    title = "philippine_{}".format(date)
    print(title)
    with open("{}.txt".format(title), "w") as text_file:
        text_file.write(text)
    
    payload = nucleus_api.Deletedatasetmodel(dataset=title)
    try:
        api_response = api_instance.post_delete_dataset(payload)
        print(api_response.result['result'])
    except ApiException as e:
        print("Dataset did not exist. (you can ignore this message)")
    
    
    api_response = api_instance.post_upload_file("{}.txt".format(title), title, metadata={"tokenization_ngram": "1"})
    fp = api_response.result
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', title,)    
    
    
    sents_bydate[date] = tokenize.sent_tokenize(text)

philippine_2019-04-22
Dataset philippine_2019-04-22 has been deleted.
philippine_2019-04-22.txt ( 20849 bytes) has been added to dataset philippine_2019-04-22
philippine_2019-04-23
Dataset philippine_2019-04-23 has been deleted.
philippine_2019-04-23.txt ( 31056 bytes) has been added to dataset philippine_2019-04-23
philippine_2019-04-24
Dataset philippine_2019-04-24 has been deleted.
philippine_2019-04-24.txt ( 8706 bytes) has been added to dataset philippine_2019-04-24
philippine_2019-04-25
Dataset philippine_2019-04-25 has been deleted.
philippine_2019-04-25.txt ( 12839 bytes) has been added to dataset philippine_2019-04-25
philippine_2019-04-26
Dataset philippine_2019-04-26 has been deleted.
philippine_2019-04-26.txt ( 3281 bytes) has been added to dataset philippine_2019-04-26
philippine_2019-04-27
Dataset philippine_2019-04-27 has been deleted.
philippine_2019-04-27.txt ( 11235 bytes) has been added to dataset philippine_2019-04-27


In [132]:
parsed_data = [cleanup(sent) for sent in all_sents]
# .apply(cleanup) 
lda_model, distributions = apply_lda(parsed_data, 8)

# Get topics

## SPCA

In [221]:
spca_topics = {}
for date in data.time.unique():
    title = "philippine_{}".format(date)
    payload = nucleus_api.Topics(dataset=title,                     
                                 num_topics=5)
    api_response = api_instance.post_topic_api(payload)   
    spca_topics[date] = api_response.result.topics

In [222]:
spca_topics[data.time.unique()[1]][2].keywords.split(";")

['buildings',
 'hit',
 'damage',
 'capital',
 'rail',
 'air',
 'infrastructure',
 'disrupting']

In [223]:
spca_topics[data.time.unique()[1]][0].keywords_weight

['0.2862',
 '0.0576',
 '0.1046',
 '0.1614',
 '0.1104',
 '0.0795',
 '0.0596',
 '0.1407']

In [224]:
spca_topics[data.time.unique()[1]][0].strength

'0.4331'

# Summarize

In [275]:
# topic
base_summaries = []
our_summaries = []
for i in range(5):
    topics = spca_topics["2019-04-23"][i].keywords.split(";")
    weights = spca_topics["2019-04-23"][i].keywords_weight
    weights = [float(w) for w in weights]
    sims = []
    for s in sents_bydate["2019-04-23"]:
        similarity = 0
        tokens = tokenize.word_tokenize(s) 
        for token in tokens:
            if token in topics:
                similarity += weights[topics.index(token)]
        sims.append(similarity)
    sims = np.array(sims)
    top_sents = np.array(sents_bydate["2019-04-23"])[np.argsort(sims)][:-16:-1]
    
    base_summary = top_sents[:5]
    base_summaries.append(list(base_summary))
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(top_sents)

    kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
    
    sampled_summary = []
    for i in range(5):
        sampled_summary.append(top_sents[list(kmeans.labels_).index(i)])
    our_summaries.append(sampled_summary)

**Similar Sentences:**

**i)**

-  'The Philippine Institute of Volcanology and Seismology reported that a magnitude 6.1 earthquake hit Castillejos Zambales at around 5:11 p. m., Monday.'
- 'The Philippine Institute of Volcanology and Seismology (Phivolcs) revised the magnitude of the earthquake to 6.1-magnitude from the earlier reported 5.7-magnitude.'

**ii)**

- 'In its initial earthquake bulletin, the Philippine Institute of Volcanology and Seismology (Phivolcs) said the earthquake struck 2 kilometers northeast of this town at 5.11 p.m.'
- 'In its initial earthquake bulletin, Philippine Institute of Volcanology and Seismology (Phivolcs) said the earthquake struck 2 kilometers northeast of this town at 5.11 p.m.'
- 'In its initial earthquake bulletin, the Philippine Institute of Volcanology and Seismology (Phivolcs) said the earthquake struck 2 kilometers northeast of this town at 5.11 p.m.'
- 'An its initial earthquake bulletin, the Philippine Institute of Volcanology and Seismology (Phivolcs) reported that the earthquake was of tectonic origin had a depth of 21 kilometers.'

**iii)**
- 'In a separate tweet, he said the operations of LRT-2 would no longer resume for the day.'
- 'LRT2 will no longer resume operations tonight.'

In [278]:
# generate the summary w/ clustering
" ".join([" ".join(s) for s in our_summaries])

# Topic-Based Metrics

we use less sparse topic model to examine topic coverage of our generated summary.

$$(Metric)=$$ 

In [280]:
spca_topics = {}
for date in data.time.unique():
    title = "philippine_{}".format(date)
    payload = nucleus_api.Topics(dataset=title,                     
                                 num_topics=10, 
                                 num_keywords=10)
    api_response = api_instance.post_topic_api(payload)   
    spca_topics[date] = api_response.result.topics
    
def topic_metric(summary):
    metric = 0
    for i in range(10):
        topics = spca_topics["2019-04-23"][i].keywords.split(";")
        weights = spca_topics["2019-04-23"][i].keywords_weight
        weights = [float(w) for w in weights]
        strength = float(spca_topics["2019-04-23"][i].strength)
        
        for j in range(len(topics)):
            if topics[j] in summary:
                metric += weights[j] * strength
                
    return metric

**Topic-based model** (base model)

In [282]:
topic_metric(" ".join([" ".join(s) for s in base_summaries]))

0.6045005

**Topic-based model with clustering** (our model)

In [281]:
topic_metric(" ".join([" ".join(s) for s in our_summaries]))

0.6264215399999999

**TextRank** 

In [300]:
topic_metric(result["2019-04-23"])

0.5839116999999998

**We observe that redundancy reduction method enables use to generate summary that has higher topic coverage, meaning that it contains more keywords from the original document (considering topic strenghts and keyword weights). By setting a constraint on the length of the document, this can indirectly penalize redundant sentences in the summarized docuemt.**

# Redundancy Metrics (with Jaccard index)

work in progress
        

## experiments (disregard them)

In [17]:
result

{'2019-04-22': 'A strong earthquake shook the Philippine capital and outlying provinces Monday, briefly knocking out power, causing boulders to roll down onto a highway, stopping an overhead train service and prompting thousands of people to flee to safety, officials said.\nThe U.S. Geological Survey said the late afternoon quake had a preliminary magnitude of 6.1 and was centered near the northwestern town of Bodega.\nThere were no immediate reports of injuries or widespread damage.Renato Solidum, who heads the government\'s earthquake institute, said the quake was not strong enough to trigger a tsunami.Gov. Lilia Pineda of Pampanga province, north of Manila, told DZMM radio that an old church was damaged and some boulders rolled down a hill onto a highway, and some concrete power posts were also toppled in one town.Thousands of office workers dashed out of buildings in Manila, many wearing hard hats, and residents ran out of houses in panic.\nSeveral towns briefly lost power on the m

In [37]:
result_short["2019-04-22"].split("\n")

['A strong earthquake shook the Philippine capital and outlying provinces Monday, briefly knocking out power, causing boulders to roll down onto a highway, stopping an overhead train service and prompting thousands of people to flee to safety, officials said.',
 "There were no immediate reports of injuries or widespread damage.Renato Solidum, who heads the government's earthquake institute, said the quake was not strong enough to trigger a tsunami.Gov. Lilia Pineda of Pampanga province, north of Manila, told DZMM radio that an old church was damaged and some boulders rolled down a hill onto a highway, and some concrete power posts were also toppled in one town.Thousands of office workers dashed out of buildings in Manila, many wearing hard hats, and residents ran out of houses in panic.",
 'Several towns briefly lost power on the main northern Luzon island, officials said.The Philippines, one of the world\'s most disaster-prone countries, is prone to earthquakes and volcanic eruptions 

In [16]:
result_short["2019-04-27"].split("\n")

['Ricardo Salad, executive director of the council, said the earthquake also left 81 people injured while 14 others remained missing in central Luzon provinces of Pampanga and Zambales.',
 "Philippine Institute of Volcanology and Seismology said Monday's earthquake was detected at a depth of 12 km about 18 km northeast of Castillejos, a town in Zambales province in the western part of Luzon.",
 'Following the first earthquake on Monday, a 6.5-magnitude earthquake struck Eastern Samar province in the central Philippines at 1.37pm local time on Tuesday.',
 'have dispatched rescue teams to help find survivors of the collapsed building in Porac, with the electric company providing the generator sets, floodlights, and heavy equipment.',
 'On Aug. 17, 1983, a 6.5-magnitude, Intensity 7 earthquake was felt in Laoag City and the municipality of Pasuquin in Ilocos Norte.',
 'A number of reinforced concrete buildings either totally crumbled or sustained major structural damage beyond rehabilitat

In [33]:
data["content"][data["time"]=="2019-04-27"] 

41    Here is the text of news released by official ...
42    Companies led by business titan Manuel V. Pang...
43    Earthquakes occur quite frequently in the Phil...
Name: content, dtype: object

In [46]:
data["content"][data["time"]=="2019-04-27"][42]

"Companies led by business titan Manuel V. Pangilinan have joined forces to provide aid in Porac, Pampanga, which was badly hit by the recent 6. 1-magnitude earthquake. PLDT provided free Fibr Link at the Command Center in Porac, which various government agencies used to communicate the strong quake's aftermath that happened last Monday.  Wireless unit Smart Communications also set up free calls and charging stations. PLDT, PLDT-Smart Foundation (PSF) and TV5's Alagang Kapatid Foundation distributed meals to responders and affected residents, while Maynilad Water Services gave bottled drinking water.  PSF also distributed 250 relief packages to families affected by the earthquake. The private sector-led Philippine Disaster Resilience Foundation (PDRF) that Pangilinan co-chairs has also deployed search and rescue teams, response coordination, logistic support, food and water, medical assistance and temporary shelters.  The efforts of the MVP consortium's Tulong Kapatid also focused on a

In [99]:
lda_model.show_topics()

[(0,
  '0.005*"persons" + 0.005*"supermarket" + 0.004*"na" + 0.004*"sa" + 0.004*"rescue" + 0.004*"collapse" + 0.003*"pdrrmc" + 0.003*"operation" + 0.003*"west" + 0.003*"inside"'),
 (1,
  '0.005*"philippines" + 0.004*"story" + 0.004*"build" + 0.003*"earthquakes" + 0.003*"province" + 0.003*"kill" + 0.003*"add" + 0.003*"northern" + 0.003*"collapse" + 0.003*"cause"'),
 (2,
  '0.003*"train" + 0.003*"service" + 0.003*"briefly" + 0.003*"boulders" + 0.003*"roll" + 0.003*"thousands" + 0.003*"prone" + 0.003*"highway" + 0.003*"power" + 0.003*"safety"'),
 (3,
  '0.005*"bataan" + 0.004*"bulacan" + 0.003*"church" + 0.003*"pampanga" + 0.003*"city" + 0.003*"outside" + 0.003*"towns" + 0.003*"see" + 0.003*"rush" + 0.003*"patients"'),
 (4,
  '0.002*"supermarket" + 0.002*"operation" + 0.002*"pdrrmc" + 0.002*"persons" + 0.002*"earthquakes" + 0.002*"fatalities" + 0.002*"west" + 0.002*"collapse" + 0.002*"province" + 0.002*"predict"'),
 (5,
  '0.006*"lrt" + 0.003*"inspection" + 0.003*"resume" + 0.003*"train" 

In [101]:
lda_model.get_topics()

array([[0.00123829, 0.0011685 , 0.00116689, ..., 0.00228238, 0.00224514,
        0.00223343],
       [0.00165542, 0.00131994, 0.00133937, ..., 0.0012411 , 0.0012411 ,
        0.0012411 ],
       [0.00152679, 0.00240512, 0.00237589, ..., 0.00129281, 0.00129542,
        0.00129477],
       ...,
       [0.00141391, 0.00132348, 0.00132348, ..., 0.00132497, 0.0013254 ,
        0.00132481],
       [0.00137561, 0.00134022, 0.00134022, ..., 0.00134022, 0.00134023,
        0.00134023],
       [0.00170205, 0.00116537, 0.00116894, ..., 0.00116319, 0.0011669 ,
        0.00116513]], dtype=float32)

In [66]:
data["content"][data["time"]=="2019-04-22"][0]

'A strong earthquake shook the Philippine capital and outlying provinces Monday, briefly knocking out power, causing boulders to roll down onto a highway, stopping an overhead train service and prompting thousands of people to flee to safety, officials said. The U.S. Geological Survey said the late afternoon quake had a preliminary magnitude of 6.1 and was centered near the northwestern town of Bodega. There were no immediate reports of injuries or widespread damage.Renato Solidum, who heads the government\'s earthquake institute, said the quake was not strong enough to trigger a tsunami.Gov. Lilia Pineda of Pampanga province, north of Manila, told DZMM radio that an old church was damaged and some boulders rolled down a hill onto a highway, and some concrete power posts were also toppled in one town.Thousands of office workers dashed out of buildings in Manila, many wearing hard hats, and residents ran out of houses in panic. An elevated train service was halted and passengers alighte

In [55]:
apply_lda(parsed_data, 4)

ValueError: cannot compute LDA over an empty collection (no terms)

In [88]:
"".join(list(data["content"][data["time"]=="2019-04-27"]))

"Here is the text of news released by official news agency Bernama on its website:  - The death toll from Monday's 6.1-magnitude earthquake that struck the northern Philippines has risen to 16, Xinhua news agency reported, quoting the Philippine National Disaster Risk Reduction and Management Council on Tuesday.  Ricardo Salad, executive director of the council, said the earthquake also left 81 people injured while 14 others remained missing in central Luzon provinces of Pampanga and Zambales.  Philippine Institute of Volcanology and Seismology said Monday's earthquake was detected at a depth of 12 km about 18 km northeast of Castillejos, a town in Zambales province in the western part of Luzon.  Following the first earthquake on Monday, a 6.5-magnitude earthquake struck Eastern Samar province in the central Philippines at 1.37pm local time on Tuesday.  No casualties and damage have been reported yet from this earthquake. Companies led by business titan Manuel V. Pangilinan have joined

In [89]:
sum_sents = tokenize.sent_tokenize(" ".join(list(data["content"][data["time"]=="2019-04-27"])))

In [90]:
sum_sents

["Here is the text of news released by official news agency Bernama on its website:  - The death toll from Monday's 6.1-magnitude earthquake that struck the northern Philippines has risen to 16, Xinhua news agency reported, quoting the Philippine National Disaster Risk Reduction and Management Council on Tuesday.",
 'Ricardo Salad, executive director of the council, said the earthquake also left 81 people injured while 14 others remained missing in central Luzon provinces of Pampanga and Zambales.',
 "Philippine Institute of Volcanology and Seismology said Monday's earthquake was detected at a depth of 12 km about 18 km northeast of Castillejos, a town in Zambales province in the western part of Luzon.",
 'Following the first earthquake on Monday, a 6.5-magnitude earthquake struck Eastern Samar province in the central Philippines at 1.37pm local time on Tuesday.',
 'No casualties and damage have been reported yet from this earthquake.',
 'Companies led by business titan Manuel V. Pangi

In [92]:
parsed_data = data["content"][data["time"]=="2019-04-22"].apply(cleanup) 
apply_lda(parsed_data, 4)

0     [strong, earthquake, shake, philippine, capita...
1     [earthquake, jolt, luzon, monday, afternoon, c...
2     [people, kill, magnitude, earthquake, strike, ...
3     [philippine, institute, volcanology, seismolog...
4     [wall, circular, fashion, strong, temblor, jol...
5     [people, kill, collapse, build, cause, magnitu...
6     [strong, magnitude, quake, record, today, phil...
7     [power, outages, hit, balanga, city, mariveles...
8     [management, train, station, suspend, operatio...
9     [flight, clark, international, airport, cancel...
10    [glass, door, fast, food, restaurant, sm, pamp...
11    [persons, confirm, die, province, result, powe...
Name: content, dtype: object

In [ ]:
data["content"][data["time"]=="2019-04-22"].apply(cleanup)

In [ ]:
apply_lda(data, num_topics)

In [58]:
num_topics=8

In [60]:
bow_corpus

[[], [], [], [], [], [], [], [], [], [], [], []]

In [104]:
lda_model.show_topic(3, topn = 10)

[('bataan', 0.0050202785),
 ('bulacan', 0.0041951407),
 ('church', 0.0030830163),
 ('pampanga', 0.0030752665),
 ('city', 0.0030282938),
 ('outside', 0.0030110118),
 ('towns', 0.0029405064),
 ('see', 0.0029222593),
 ('rush', 0.0028828823),
 ('patients', 0.0028442738)]

In [102]:
sum([a[1] for a in lda_model.show_topic(0, topn = 10)])

0.037422470515593886

In [98]:
lda_model.show_topic(1, topn = 10)

[('philippines', 0.0046483115),
 ('story', 0.0042084563),
 ('build', 0.0039967415),
 ('earthquakes', 0.003482062),
 ('province', 0.003374999),
 ('kill', 0.003223483),
 ('add', 0.0031926653),
 ('northern', 0.0030283232),
 ('collapse', 0.0029627318),
 ('cause', 0.002933765)]

In [95]:
distributions

[{1: 0.07745821, 2: 0.8305603, 7: 0.086699255},
 {0: 0.67082036, 3: 0.19629927, 7: 0.12882827},
 {1: 0.6193342, 7: 0.37458244},
 {0: 0.9679812, 7: 0.028356703},
 {0: 0.096097074, 1: 0.06755608, 3: 0.012471877, 6: 0.22302645, 7: 0.5920847},
 {1: 0.9696112, 7: 0.022517633},
 {7: 0.9794977},
 {3: 0.45974904, 7: 0.5331422},
 {5: 0.9835608, 7: 0.010368145},
 {3: 0.8031328, 7: 0.19336322},
 {7: 0.9872511},
 {0: 0.98975796}]

In [40]:
def cleanup(doc):
    result = []
    doc = re.sub(re.compile('<.*?>'), '', doc)
    for word in gensim.utils.simple_preprocess(doc):
        if word not in gensim.parsing.preprocessing.STOPWORDS and len(doc) > 3:
            result.append(WordNetLemmatizer().lemmatize(word, pos='v'))
    return result

def apply_lda(data, num_topics):
    dictionary = gensim.corpora.Dictionary(data)
#     dictionary.filter_extremes(no_below=10, no_above=0.4)
    bow_corpus = [dictionary.doc2bow(text_lst) for text_lst in data]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=2, workers=4, iterations=10)
    distributions = [dict(lda_model[sentence]) for sentence in bow_corpus]
    return lda_model, distributions

# Cluster points

In [202]:
sum_sents = tokenize.sent_tokenize(result["2019-04-27"])

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sum_sents)

kmeans = KMeans(n_clusters=8, random_state=0).fit(X)
kmeans.labels_

array([6, 2, 5, 1, 1, 2, 7, 1, 7, 2, 1, 2, 7, 7, 2, 1, 3, 3, 2, 3, 0, 1,
       7, 3, 0, 4, 3, 2, 3, 7, 3, 1, 2, 0, 2, 2, 2, 7], dtype=int32)

In [68]:
sum_sents

["Here is the text of news released by official news agency Bernama on its website:  - The death toll from Monday's 6.1-magnitude earthquake that struck the northern Philippines has risen to 16, Xinhua news agency reported, quoting the Philippine National Disaster Risk Reduction and Management Council on Tuesday.",
 'Ricardo Salad, executive director of the council, said the earthquake also left 81 people injured while 14 others remained missing in central Luzon provinces of Pampanga and Zambales.',
 "Philippine Institute of Volcanology and Seismology said Monday's earthquake was detected at a depth of 12 km about 18 km northeast of Castillejos, a town in Zambales province in the western part of Luzon.",
 'Following the first earthquake on Monday, a 6.5-magnitude earthquake struck Eastern Samar province in the central Philippines at 1.37pm local time on Tuesday.',
 "PLDT provided free Fibr Link at the Command Center in Porac, which various government agencies used to communicate the st

In [27]:
np.array(sum_sents)[kmeans.labels_==2]

array(['Following the first earthquake on Monday, a 6.5-magnitude earthquake struck Eastern Samar province in the central Philippines at 1.37pm local time on Tuesday.',
       'On Aug. 17, 1983, a 6.5-magnitude, Intensity 7 earthquake was felt in Laoag City and the municipality of Pasuquin in Ilocos Norte.',
       'On Feb. 8, 1990, a 6.8-magnitude earthquake, which was felt at Intensity 8, hit the municipalities of Jagna, Duero and Guindulman in Bohol.',
       'Two days later, a 5.0-magnitude earthquake was felt at Intensity 6 in Talacogon, Agusan del Sur.',
       'On Oct. 15, 2013, the 7.2-magnitude earthquake, felt at Intensity 7, in Tagbilaran City, Bohol caused major destruction in the province.'],
      dtype='<U204')